In [14]:
%tensorflow_version 1.x
!pip install adversarial-robustness-toolbox
!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans/

fatal: destination path 'cleverhans' already exists and is not an empty directory.
Processing ./cleverhans
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=253453 sha256=65950bd868c9f7dfd5c335c787d2d0faf86a39976c04141888c31b5c83ac7fd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-djnxtjbj/wheels/d1/6b/1d/5cf7b3ca4c0cfc7f845628b8ed46366ab5f4f56b5483e9db7f
Successfully built cleverhans
  Found existing installation: cleverhans 3.0.1
    Uninstalling cleverhans-3.0.1:
      Successfully uninstalled cleverhans-3.0.1


In [0]:
import tensorflow_hub as thub

In [3]:
# most of our imports
import warnings
import numpy as np
import os
with warnings.catch_warnings():
    import keras # keras is still using some deprectade code
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.attacks import BasicIterativeMethod, FastGradientMethod, CarliniWagnerL2
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
from art.classifiers import KerasClassifier

Using TensorFlow backend.


In [0]:
def convert_to_keras_image_format( x_train, x_test ):
    if keras.backend.image_data_format( ) == 'channels_first':
        x_train = x_train.reshape( x_train.shape[ 0 ], 1, x_train.shape[ 1 ], x_train.shape[ 2 ] )
        x_test = x_test.reshape( x_test.shape[ 0 ], 1, x_train.shape[ 1 ], x_train.shape[ 2 ] )
    else:
        x_train = x_train.reshape( x_train.shape[ 0 ], x_train.shape[ 1 ], x_train.shape[ 2 ], 1 )
        x_test = x_test.reshape( x_test.shape[ 0 ], x_train.shape[ 1 ], x_train.shape[ 2 ], 1 )

    return x_train, x_test

In [0]:
# # most of our imports
# import warnings
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow as tf
import tensorflow_hub as thub
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

# from keras.models import Model
# from keras.layers import Input
# from keras import Sequential
# from keras.optimizers import SGD
# from keras.losses import CategoricalCrossentropy
# from keras.utils import to_categorical
# from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# from art.classifiers import KerasClassifier

# from lib.data_utils import *

In [0]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod

In [0]:
from keras.datasets import cifar10

In [24]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
testd = x_train[0:1000]/255
testl = y_train[0:1000]

In [0]:
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

gan_eval_layer = thub.KerasLayer(model_url)

In [18]:
gan_eval_model = tf.keras.Sequential([tf.keras.layers.InputLayer(input_shape=(32,32,3)),gan_eval_layer,tf.keras.Activation()])
# gan_eval_model = Sequential([gan_eval_layer])
# gan_eval_model.build(input_shape=(None,32,32,3))

In [19]:
gan_eval_model.compile(
  optimizer=SGD(lr=0.1, momentum=0.9, decay=1e-4),
  loss=CategoricalCrossentropy(),
  metrics=['accuracy'])
gan_eval_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 10)                7796426   
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [0]:
classifier = KerasClassifier(model=gan_eval_model, clip_values=(0, 1), use_logits=True)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.3)

In [0]:
x_test_adv = attack_fgsm.generate(testd)

In [39]:
gan_eval_model.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 11s 11ms/sample - loss: 1.1921e-07 - acc: 1.0000


[1.1920928955078125e-07, 1.0]

In [33]:
testl

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
lt = gan_eval_model.predict(x_test_adv)

In [41]:
lt

array([[-2.4097886 , -3.1525166 , -1.6822641 , ...,  1.4174325 ,
        -5.4198    , -2.9038186 ],
       [-1.9242427 ,  0.77200556, -2.3829396 , ..., -2.9516826 ,
         1.2336738 , 12.365367  ],
       [ 0.6440714 , -1.5064375 ,  2.2270122 , ..., -0.9539639 ,
         0.7637255 , 11.43376   ],
       ...,
       [-1.6161097 , 13.830811  , -1.5407586 , ..., -3.7686806 ,
         2.415141  ,  1.8893585 ],
       [ 2.3144345 , -1.8472934 ,  0.5314843 , ..., -1.0430295 ,
        -1.3715537 , -3.2575228 ],
       [-4.9218826 , -0.9399113 ,  0.02922496, ..., -1.741168  ,
        -2.3313975 , -1.965477  ]], dtype=float32)